# CS558 Homework #1 Edge Detection
Author: Matthew Halvorsen

In [1]:
import numpy as np
from PIL import Image
import os

Question 1

In [2]:
def gaussian_filter(image_path, sigma):
    # Load the image and convert to grayscale/array
    img = Image.open(image_path).convert('L')
    img_arr = np.array(img, dtype=float)
    
    # Determine kernel size (6*sigma rule ensures sum is ~1)
    # Must be odd to have a center pixel
    size = int(6 * sigma + 1)
    if size % 2 == 0:
        size += 1
    
    center = size // 2
    
    # Create the Gaussian Kernel
    # Create a coordinate grid
    x, y = np.mgrid[-center:center+1, -center:center+1]
    
    # Apply the 2D Gaussian formula
    # G(x,y) = (1 / (2 * pi * sigma^2)) * exp(-(x^2 + y^2) / (2 * sigma^2))
    kernel = np.exp(-(x**2 + y**2) / (2 * sigma**2))
    kernel /= kernel.sum()  # Normalize so the sum is exactly 1
    
    # Manual Convolution
    h, w = img_arr.shape
    new_img = np.zeros_like(img_arr)
    
    # Pad the image to handle borders (zero padding)
    padded_img = np.pad(img_arr, center, mode='constant')
    
    for i in range(h):
        for j in range(w):
            # Extract the region of interest (ROI)
            roi = padded_img[i:i+size, j:j+size]
            # Element-wise multiplication and sum
            new_img[i, j] = np.sum(roi * kernel)
            
    return Image.fromarray(new_img.astype(np.uint8))

This is only set up to run the gausain filter on the kangaroo image with two different sigmas. The image path needs to be chagned if other images are to be used.

In [3]:
path_kang = "kangaroo.pgm"
filtered_img_high = gaussian_filter(path_kang, sigma=7)
filtered_img_high.show()
filtered_img_high.save('Gaussian filter high threshold.png')

filtered_img_low = gaussian_filter(path_kang, sigma=2)
filtered_img_high.show()
filtered_img_high.save('Gaussian filter low threshold.png')

Question 2

In [4]:
def sobel_edge_detection(image_path, threshold=100):
    # Load and convert to grayscale
    img = Image.open(image_path).convert('L')
    img_arr = np.array(img, dtype=float)
    
    # Sobel kernels
    Kx = np.array([[-1, 0, 1], 
                   [-2, 0, 2], 
                   [-1, 0, 1]])
    
    Ky = np.array([[-1, -2, -1], 
                   [ 0,  0,  0], 
                   [ 1,  2,  1]])
    
    h, w = img_arr.shape
    gx = np.zeros_like(img_arr)
    gy = np.zeros_like(img_arr)

    padded_img = np.pad(img_arr, 1, mode='edge')
    
    # Convolution
    for i in range(h):
        for j in range(w):
            roi = padded_img[i:i+3, j:j+3]
            gx[i, j] = np.sum(roi * Kx)
            gy[i, j] = np.sum(roi * Ky)
    
    # Magnitude
    magnitude = np.sqrt(gx**2 + gy**2)

    # Normalize magnitude for display
    magnitude_norm = (magnitude / (magnitude.max() + 1e-12)) * 255
    magnitude_norm = magnitude_norm.astype(np.uint8)

    # Thresholding edge map
    edge_map = np.where(magnitude_norm > threshold, 255, 0).astype(np.uint8)

    # Return images + gradients
    return Image.fromarray(magnitude_norm), Image.fromarray(edge_map), gx, gy, magnitude

Each block is set up to run the sobel edge detector on each image

In [5]:
mag_img_plane, edge__img_plane, gx_plane, gy_plane, mag_plane = sobel_edge_detection("plane.pgm", threshold=80)

mag_img_plane.show()
edge__img_plane.show()
mag_img_plane.save("sobel_magnitude_plane.png")
edge__img_plane.save("sobel_edges_plane.png")

In [6]:
mag_img_kang, edge__img_kang, gx_kang, gy_kang, mag_kang = sobel_edge_detection("kangaroo.pgm", threshold=60)

mag_img_kang.show()
edge__img_kang.show()
mag_img_kang.save("sobel_magnitude_kang.png")
edge__img_kang.save("sobel_edges_kang.png")

In [7]:
mag_img_red, edge__img_red, gx_red, gy_red, mag_red = sobel_edge_detection("red.pgm", threshold=60)

mag_img_red.show()
edge__img_red.show()
mag_img_red.save("sobel_magnitude_red.png")
edge__img_red.save("sobel_edges_red.png")

Question 3

In [8]:
def non_max_suppression(mag: np.ndarray, gx: np.ndarray, gy: np.ndarray) -> np.ndarray:
    H, W = mag.shape
    nms = np.zeros_like(mag, dtype=mag.dtype)

    # Compute gradient direction (0 to 180 degrees)
    angle = np.rad2deg(np.arctan2(gy, gx)) % 180.0

    # Quantize direction into 4 bins: 0, 45, 90, 135
    q = np.zeros_like(angle, dtype=np.uint8)
    q[(angle >= 22.5) & (angle < 67.5)] = 45
    q[(angle >= 67.5) & (angle < 112.5)] = 90
    q[(angle >= 112.5) & (angle < 157.5)] = 135
    # else remains 0

    # Loop through pixels (ignore border pixels)
    for y in range(1, H - 1):
        for x in range(1, W - 1):
            m = mag[y, x]
            direction = q[y, x]

            # Pick neighbors based on gradient direction
            if direction == 0:
                n1, n2 = mag[y, x - 1], mag[y, x + 1]
            elif direction == 45:
                n1, n2 = mag[y - 1, x + 1], mag[y + 1, x - 1]
            elif direction == 90:
                n1, n2 = mag[y - 1, x], mag[y + 1, x]
            else:  # 135
                n1, n2 = mag[y - 1, x - 1], mag[y + 1, x + 1]

            # Keep only local maxima
            if (m >= n1) and (m >= n2):
                nms[y, x] = m
            else:
                nms[y, x] = 0

    return nms

Each block is set up to run for each image in the dataset

In [9]:
nms_kang = non_max_suppression(mag_kang, gx_kang, gy_kang)

# normalize for saving
nms_img_kang = (nms_kang / (nms_kang.max() + 1e-12) * 255).astype(np.uint8)
Image.fromarray(nms_img_kang).save("nms_kang.png")

In [10]:
nms_plane = non_max_suppression(mag_plane, gx_plane, gy_plane)

# normalize for saving
nms_img_plane = (nms_plane / (nms_plane.max() + 1e-12) * 255).astype(np.uint8)
Image.fromarray(nms_img_plane).save("nms_plane.png")

In [11]:
nms_red = non_max_suppression(mag_red, gx_red, gy_red)

# normalize for saving
nms_img_red = (nms_red / (nms_red.max() + 1e-12) * 255).astype(np.uint8)
Image.fromarray(nms_img_red).save("nms_red.png")